In [1]:
import os
import json
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from pypfopt import EfficientFrontier, risk_models, expected_returns, plotting, DiscreteAllocation, objective_functions

In [2]:
# -------------------------------
# Config
# -------------------------------
MODELS_DIR = os.path.join("..", "models")         # same location you used before
FORECAST_CSV = os.path.join(MODELS_DIR, "tsla_lstm_forecast_mc_dropout.csv")
METADATA_FILE = os.path.join(MODELS_DIR, "tsla_lstm_metadata.json")

TICKER_TSLA = "TSLA"
TICKERS_OTHER = ["BND", "SPY"]
ALL_TICKERS = [TICKER_TSLA] + TICKERS_OTHER

TRADING_DAYS = 252    # used for annualization

In [4]:
# -------------------------------
# Load metadata & forecast
# -------------------------------
if not os.path.exists(FORECAST_CSV):
    raise FileNotFoundError(f"Forecast CSV not found at {FORECAST_CSV}. Run Task 3 first.")

with open(METADATA_FILE, "r") as f:
    metadata = json.load(f)

# Historical data range used for training (metadata)
hist_start = metadata.get("start_date", "2015-07-01")
hist_end = metadata.get("end_date", "2025-07-31")

# Forecasted TSLA mean series
forecast_df = pd.read_csv(FORECAST_CSV, parse_dates=["date"], index_col="date")
if "forecast_mean" not in forecast_df.columns:
    raise KeyError("forecast_mean column not found in forecast CSV.")

# Compute forecast-based expected return for TSLA:
# Approach: compute average daily forecast return over the forecast horizon, then annualize.
forecast_prices = forecast_df["forecast_mean"].values  # array length = FORECAST_DAYS
# compute daily returns of the forecast sequence: r_t = (p_t / p_{t-1}) - 1
# For first step, use last historical price as previous price
# load last historical price from yfinance to ensure alignment
hist_full = yf.download(TICKER_TSLA, start=hist_start, end=hist_end)
last_hist_price = hist_full["Close"].dropna().iloc[-1]
# build an augmented array with last_hist_price followed by forecast prices
aug_prices = np.concatenate([[last_hist_price], forecast_prices])
forecast_daily_returns = (aug_prices[1:] / aug_prices[:-1]) - 1.0
# expected daily return from forecast (mean)
tsla_exp_daily = np.nanmean(forecast_daily_returns)
tsla_exp_annual = (1 + tsla_exp_daily) ** TRADING_DAYS - 1  # compound to annual

C:\Users\hp\AppData\Local\Temp\ipykernel_14680\3620603927.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  hist_full = yf.download(TICKER_TSLA, start=hist_start, end=hist_end)
[*********************100%***********************]  1 of 1 completed

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [5]:
# -------------------------------
# Load historical price data for BND and SPY (use same historical window)
# -------------------------------
print(f"Fetching historical data for {', '.join(TICKERS_OTHER)} from {hist_start} to {hist_end} ...")
df_other = yf.download(TICKERS_OTHER, start=hist_start, end=hist_end)["Close"].dropna()
# compute daily returns
returns_other = df_other.pct_change().dropna()
# historical expected returns (annualized mean of daily returns)
exp_returns_other_annual = (1 + returns_other.mean()) ** TRADING_DAYS - 1
# convert to dict-like
exp_returns_other_annual = exp_returns_other_annual.to_dict()

Fetching historical data for BND, SPY from 2015-07-01 to 2025-07-31 ...


C:\Users\hp\AppData\Local\Temp\ipykernel_14680\965560297.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_other = yf.download(TICKERS_OTHER, start=hist_start, end=hist_end)["Adj Close"].dropna()
[*********************100%***********************]  2 of 2 completed


KeyError: 'Adj Close'

In [ ]:
# -------------------------------
# Build expected returns vector (annual)
# -------------------------------
# TSLA from forecast, BND & SPY from historical mean
mu_annual = pd.Series({
    TICKER_TSLA: tsla_exp_annual,
    "BND": exp_returns_other_annual.get("BND", 0.0),
    "SPY": exp_returns_other_annual.get("SPY", 0.0)
})

print("\nExpected annual returns (used in optimization):")
print(mu_annual.round(6))